# Prep

In [44]:
import pandas as pd
from tqdm import tqdm


df = pd.read_csv('df_preprocessed.csv')
df = df.drop(df.columns[:2], axis = 1)

import ast
df['poem'] = df['poem'].apply(ast.literal_eval)
df['poem_lemma'] = df['poem_lemma'].apply(ast.literal_eval)


hun_stopwords = []
with open('stopwords.txt','r') as o:
    for line in o:
        hun_stopwords.append(line)
import re
hun_stopwords = [re.sub("\n", "", x) for x in hun_stopwords]

[hun_stopwords.append(x) for x in 
    ['hogyha', 'szó', 'mond', 'akar', 'oly', 'beszéd', 
    'monda', 'miként', 'beszél', 'féle', 'vajha', 'amiképp',
    'vesz', 'tesz', 'dal', 'szem']]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [45]:
df['poem_lemma'] = [" ".join([y for y in x if y not in hun_stopwords]) for x in df.poem_lemma]
df = df[['poet', 'poem_lemma']]

In [6]:
import spacy
import huspacy

In [66]:
train_df = df.groupby('poet', group_keys=False).apply(lambda x: x.sample(frac=0.77, random_state=42))
dev_df = df.drop(train_df.index).apply(lambda x: x.sample(frac=0.6, random_state=42))
test_df = df.drop(train_df.index).drop(dev_df.index)


<positron-console-cell-66>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [67]:
nlp = spacy.blank('hu')

In [68]:
from spacy.tokens import DocBin

def convert_from_df(df,outfile, text_column = 'poem_lemma', label_column = 'poet'):
    db = DocBin()
    
    for text, label in zip(df[text_column], df[label_column]):
        doc = nlp.make_doc(text)
        # Initialize all categories to False
        doc.cats = {poet: False for poet in df[label_column].unique()}
        # Set the current label category to True
        doc.cats[label] = True
        db.add(doc)
    
    db.to_disk(outfile)

poet_labels = df.poet.unique()

convert_from_df(train_df, outfile="./train.spacy")
convert_from_df(dev_df,  outfile="./dev.spacy")
convert_from_df(test_df, outfile="./test.spacy")

In [69]:
!python -m spacy init config  --lang hu --pipeline textcat --optimize efficiency --force config.cfg

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: hu
- Pipeline: textcat
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: hu
- Pipeline: textcat
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [70]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output model --verbose

[2024-12-02 18:00:16,890] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: model
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2024-12-02 18:00:16,890] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: model
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================


[2024-12-02 18:00:17,028] [INFO] Set up nlp object from config
[2024-12-02 18:00:17,036] [DEBUG] Loading corpus from path: dev.spacy
[2024-12-02 18:00:17,037] [DEBUG] Loading corpus from path: train.spacy
[2024-12-02 18:00:17,037] [INFO] Pipeline: ['textcat']
[2024-12-02 18:00:17,039] [INFO] Created vocabulary
[2024-12-02 18:00:17,039] [INFO] Finished initializing nlp object
[2024-12-02 18:00:17,028] [INFO] Set up nlp object from config
[2024-12-02 18:00:17,036] [DEBUG] Loading corpus from path: dev.spacy
[2024-12-02 18:00:17,037] [DEBUG] Loading corpus from path: train.spacy
[2024-12-02 18:00:17,037] [INFO] Pipeline: ['textcat']
[2024-12-02 18:00:17,039] [INFO] Created vocabulary
[2024-12-02 18:00:17,039] [INFO] Finished initializing nlp object


[2024-12-02 18:00:30,685] [INFO] Initialized pipeline components: ['textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
[2024-12-02 18:00:30,693] [DEBUG] Loading corpus from path: dev.spacy
[2024-12-02 18:00:30,685] [INFO] Initialized pipeline components: ['textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
[2024-12-02 18:00:30,693] [DEBUG] Loading corpus from path: dev.spacy


[2024-12-02 18:00:30,694] [DEBUG] Loading corpus from path: train.spacy
[2024-12-02 18:00:30,696] [DEBUG] Removed existing output directory: model/model-best
[2024-12-02 18:00:30,698] [DEBUG] Removed existing output directory: model/model-last
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
[2024-12-02 18:00:30,694] [DEBUG] Loading corpus from path: train.spacy
[2024-12-02 18:00:30,696] [DEBUG] Removed existing output directory: model/model-best
[2024-12-02 18:00:30,698] [DEBUG] Removed existing output directory: model/model-last
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------


  0       0          0.07        1.55    0.02
  0       0          0.07        1.55    0.02


...

  4    4000          0.23       80.34    0.80
  4    4000          0.23       80.34    0.80


  5    4200          0.34       80.23    0.80
  5    4200          0.34       80.23    0.80


  5    4400          0.38       79.78    0.80
  5    4400          0.38       79.78    0.80


  6    4600          0.28       79.68    0.80
  6    4600          0.28       79.68    0.80


✔ Saved pipeline to output directory
model/model-last
✔ Saved pipeline to output directory
model/model-last


In [73]:
!python -m spacy evaluate ./model/model-best/ ./test.spacy

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0



================================== Results ==================================

TOK                 100.00
TEXTCAT (macro F)   74.44 
SPEED               247960


=========================== Textcat F (per label) ===========================

                   P       R       F
Ady            81.20   89.62   85.20
AranyJ         72.09   81.58   76.54
Babits         68.29   62.22   65.12
Balassi       100.00   77.78   87.50
Csokonai       94.44   85.00   89.47
Jozsef         81.13   74.14   77.48
Karinthy        0.00    0.00    0.00
Kolcsey       100.00   83.33   90.91
Kosztolanyi    72.86   82.26   77.27
Madach        100.00   66.67   80.00
Petofi         90.54   84.81   87.58
Radnoti        84.62   64.71   73.33
Vorosmarty     66.25   92.98   77.37


======================== Textcat ROC AUC (per label) ========================

              ROC AUC
Ady              0.97
AranyJ           0.98
Babits           0.94
Balassi          1.00
Csokonai         0.98
Jozsef           0.97
Karin

In [74]:
nlp = spacy.load("./model/model-best")

# Teszt

## Ady: Nyárdélutáni hold rómában

In [75]:
doc = nlp("""Sandítva száll Rómára
Fecske-raj-követséggel,
Vigyorog vígan széjjel
Nyárdélutáni Hold.
Nagy kékség és pirosság
Most újból-újból hozzák
Régből azt, ami volt.
Szent mezők pára-fátylat,
Hegyek álom-színt váltnak,
Diadalok s romok,
Nap s Hold közé beszőve,
Hanyattan az Időbe
Róma sürög-forog.
Óh, gyönyörű örökség,
Változó, ős, szent község,
Urbs, te feledtető,
Az én-élet poklából
Lelkem-testem kilábol,
Te szent, védő tető.

Ím, magam idehoztam,
Védj és boríts be mostan,
Te szép, te bölcs, örök.
Örökkön éltem, élek,
Csupán hüvelyt cserélek,
Mint Ulisszes-görög.
Áldom a nyüzsgő Rómát,
Mindennek átfogóját,
Pulyásan is nagyot.
Ma, hogyha úgy akarnám,
Alkonyi álom karján
Akár Remus vagyok.
Nézem a mai nőket,
A volt és lesz időket:
Be régen élek itt
S be minden élet mindegy
S a Hold már ismer minket,
Vigyorog s nem hevít:
Sandítva száll Rómára.""")



print(sorted(doc.cats.items(), key=lambda item: item[1], reverse=True))

[('Ady', 0.6256720423698425), ('Babits', 0.18055830895900726), ('Jozsef', 0.05460432171821594), ('Vorosmarty', 0.0539250522851944), ('Petofi', 0.03594687953591347), ('Kosztolanyi', 0.01609889231622219), ('Kolcsey', 0.00993380881845951), ('Csokonai', 0.008644383400678635), ('Madach', 0.007531622424721718), ('AranyJ', 0.003602896584197879), ('Balassi', 0.001917822752147913), ('Radnoti', 0.0008366082911379635), ('Karinthy', 0.0007273107185028493)]
[('Ady', 0.6256720423698425), ('Babits', 0.18055830895900726), ('Jozsef', 0.05460432171821594), ('Vorosmarty', 0.0539250522851944), ('Petofi', 0.03594687953591347), ('Kosztolanyi', 0.01609889231622219), ('Kolcsey', 0.00993380881845951), ('Csokonai', 0.008644383400678635), ('Madach', 0.007531622424721718), ('AranyJ', 0.003602896584197879), ('Balassi', 0.001917822752147913), ('Radnoti', 0.0008366082911379635), ('Karinthy', 0.0007273107185028493)]


In [76]:
doc = nlp("""
A MAGYAR NÉP
	 	

Szabad a magyar nép, szabad valahára,
Kinek láncot vertek kezére, lábára,
S görbedt derekával a rabigát vonta,
Mintha csak állat és nem ember lett volna.

Szabad a magyar nép, fejét föltarthatja,
Kénye-kedve szerint kezeit mozgatja,
S mely előbb mint bilincs őtet szorította,
A vasat mint kardot ő szorítja mostan.

Szabad a magyar nép... lejárt napod, német!
Nem táncoltatod te többé ezt a népet,
S pióca módjára nem szívod a vérét,
Megfizette isten gonoszságod bérét.

E földön legyen úr a tót vagy a német?
E földön, hol annyi vitéz magyar vérzett!
Magyar vér szerezte ezt a dicső hazát,
És magyar vér ezer évig ótalmazá!

Nincs itt urasága csak az egy magyarnak,
S kik a mi fejünkre állani akarnak,
Azoknak mi állunk feje tetejére,
S vágjuk sarkantyúnkat szíve közepébe!

Vigyázz, magyar, vigyázz, éjjel is ébren légy,
Ki tudja, mikor üt rajtad az ellenség?
Ha eljön, ugy jőjön, hogy készen találjon,
Még a félhalott se maradjon az ágyon!

Haza és szabadság, ez a két szó, melyet
Először tanuljon dajkától a gyermek,
És ha a csatában a halál eléri,
Utószor e két szót mondja ki a férfi!""")



print(sorted(doc.cats.items(), key=lambda item: item[1], reverse=True))

[('Petofi', 0.6059380173683167), ('Vorosmarty', 0.1437848061323166), ('Ady', 0.08475317806005478), ('AranyJ', 0.052388403564691544), ('Jozsef', 0.041521597653627396), ('Babits', 0.029419589787721634), ('Csokonai', 0.023742645978927612), ('Madach', 0.009019370190799236), ('Kosztolanyi', 0.008703101426362991), ('Kolcsey', 0.00030073453672230244), ('Karinthy', 0.00028921494958922267), ('Radnoti', 8.483553392579779e-05), ('Balassi', 5.458527448354289e-05)]
[('Petofi', 0.6059380173683167), ('Vorosmarty', 0.1437848061323166), ('Ady', 0.08475317806005478), ('AranyJ', 0.052388403564691544), ('Jozsef', 0.041521597653627396), ('Babits', 0.029419589787721634), ('Csokonai', 0.023742645978927612), ('Madach', 0.009019370190799236), ('Kosztolanyi', 0.008703101426362991), ('Kolcsey', 0.00030073453672230244), ('Karinthy', 0.00028921494958922267), ('Radnoti', 8.483553392579779e-05), ('Balassi', 5.458527448354289e-05)]
